### Importing

In [1]:
import pandas as pd
from impala.dbapi import connect
from impala.util import as_pandas
import time

### Connecting to impala

In [7]:
# Establishing connection

conn = connect(port = 2324, database = 'cdr')
# conn = connect(host = 'bdl3.eng.tau.ac.il', database = 'cdr')

# Creating cursor
cur = conn.cursor()

### Getting a sapmle of unchecked imsi 

In [10]:
# Getting the imsi list
imsi_list = pd.read_csv('../data/imsi_list - full.csv')

# Getting a sample of unchecked imsi
users = imsi_list[imsi_list.checked == False].sample(2000).imsi.values

# Creating a string of the imsi sapmle for the query
sample = str(tuple(users))

### Getting samples for phone calls analysis

In [3]:
# Getting the imsi list
imsi_list = pd.read_csv('../data/imsi_list - full.csv')

### Executing the query

In [13]:
# Getting the start time
start_time = time.time() 
print('Strat time: ' + time.ctime())

Strat time: Tue Aug 15 17:50:35 2017


In [14]:
# query = 'SELECT imsi, globalidx, halfhouridx, latitude, longtitude FROM traces WHERE imsi in ' + sample)
query = 'SELECT COUNT (date_stamp), MIN(date_stamp), MAX(date_stamp) FROM traces;'
# query = 'SELECT * FROM traces LIMIT 100;'

In [15]:
cur.execute(query)
data = as_pandas(cur)

In [16]:
data

,count(date_stamp),min(date_stamp),max(date_stamp)
0,16826157036,2012-11-28,2013-02-03


In [17]:
# Getting the end time
end_time = time.time() 

# Getting thr run time in hours
run_time = (end_time - start_time) / 3600

# Getting the minutes and seconds
hours = int(run_time)
minutes = int((run_time - hours) * 60)
seconds  = int((((run_time - hours) * 60) - minutes)*60)

print('End time: ' + time.ctime())
print('Run time: ' + str(hours) + ' hours ' + str(minutes) + ' minutes ' + str(seconds) + ' seconds')

End time: Tue Aug 15 17:59:09 2017
Run time: 0 hours 8 minutes 33 seconds


In [26]:
print ('{:,d}'.format(data.iloc[0,0]))

16,826,157,036


### Saving te data

In [11]:
data.to_csv('../data/samples/east_jerus.csv', index = False)

In [26]:
df = data.groupby('imsi').count()
df.globalidx.mean()

804.8654370059398

### Updating the checked imsi in the imsi list

In [ ]:
# Getting the full imsi list
imsi_list_full = pd.read_csv('../data/imsi_list - full.csv')

# Updating the current sample in the full imsi list
imsi_list_full.loc[imsi_list_full['imsi'].isin(users), 'checked'] = True

In [ ]:
# Exporting the updated file
imsi_list_full.to_csv('./data/imsi_list - full.csv', index = False)

In [ ]:
df = pd.read_csv('./')